# Preparando os dados

O objetivo deste notebook é apresentar operações que devem ser feitas em arquivos de dados( DataFrames e etc) para estes poderem ser utilizados por modelos do pacote MLJ. 

In [1]:
using Queryverse
using MLJ

primeiro carregamos um csv com o comando __load__ do pacote __Queryverse__.

___Obs: é importante notar que em todo o tutorial estamos considerando que cada coluna do arquivo é uma classe do conjunto de dados, quando for fazer uso dos dados verifique se sua classe está nas linhas ou colunas e faça os devidos ajustes.___

In [2]:
dados = Queryverse.load("iris.csv") |> DataFrame
first(dados, 6)

,sepallength,sepalwidth,petallength,petalwidth,variety
,Float64,Float64,Float64,Float64,String
1,5.1,3.5,1.4,0.2,Setosa
2,4.9,3.0,1.4,0.2,Setosa
3,4.7,3.2,1.3,0.2,Setosa
4,4.6,3.1,1.5,0.2,Setosa
5,5.0,3.6,1.4,0.2,Setosa
6,5.4,3.9,1.7,0.4,Setosa


Após isso, vamos verificar os tipos científicos de cada uma das colunas, isso é necessário poiso pacote MLJ faz uso desses tipos. Para fazermos essa verificação usamos o comando __schema__ do pacote __MLJ__. É importante saber que temos a seguinte árvore para os tipos científicos:
```
Found
├─ Known
│  ├─ Finite
│  │  ├─ Multiclass
│  │  └─ OrderedFactor
│  ├─ Infinite
│  │  ├─ Continuous
│  │  └─ Count
│  ├─ Image
│  │  ├─ ColorImage
│  │  └─ GrayImage
|  ├─ Textual
│  └─ Table
└─ Unknown
```


In [3]:
schema(dados)

┌─────────────┬─────────┬────────────┐
│ _.names     │ _.types │ _.scitypes │
├─────────────┼─────────┼────────────┤
│ sepallength │ Float64 │ Continuous │
│ sepalwidth  │ Float64 │ Continuous │
│ petallength │ Float64 │ Continuous │
│ petalwidth  │ Float64 │ Continuous │
│ variety     │ String  │ Textual    │
└─────────────┴─────────┴────────────┘
_.nrows = 150


Aqui temos que os valores de comprimento e largura foram avaliados como __Continuous__ já o parâmetro variety foi avaliado como __Textual__ o que não é desejado aqui já que ele será nossa classe, então, o mais adequado aqui é transformar ele em um __Multiclass__. Para transformarmos o tipo científico do nosso dado usaremos o comando __coerce__ do pacote __MLJ__. Para mais informações sobre os tipo científicos uma boa fonte é a [documentação](https://alan-turing-institute.github.io/MLJScientificTypes.jl/dev/#Getting-started-1).

In [4]:
coerce!(dados, :variety => Multiclass)
first(dados,6)

,sepallength,sepalwidth,petallength,petalwidth,variety
,Float64,Float64,Float64,Float64,Categorical…
1,5.1,3.5,1.4,0.2,Setosa
2,4.9,3.0,1.4,0.2,Setosa
3,4.7,3.2,1.3,0.2,Setosa
4,4.6,3.1,1.5,0.2,Setosa
5,5.0,3.6,1.4,0.2,Setosa
6,5.4,3.9,1.7,0.4,Setosa


Usando o __schema__ novamente agora vemos que o tipo da __variety__ agora está adequada, esses dados agora poderiam ser em um modelo de classificação.

In [5]:
schema(dados)

┌─────────────┬───────────────────────────┬───────────────┐
│ _.names     │ _.types                   │ _.scitypes    │
├─────────────┼───────────────────────────┼───────────────┤
│ sepallength │ Float64                   │ Continuous    │
│ sepalwidth  │ Float64                   │ Continuous    │
│ petallength │ Float64                   │ Continuous    │
│ petalwidth  │ Float64                   │ Continuous    │
│ variety     │ CategoricalString{UInt32} │ Multiclass{3} │
└─────────────┴───────────────────────────┴───────────────┘
_.nrows = 150


Existe uma função chamada __autotype__ do pacote __MLJ__ que procura os tipos mais adequados para cada categoria de forma automática. Essa função pode ser utilizada em conjunto com o __coerce__ para mudar automaticamente o tipo dos dados. Vamos carregar novamente o conjunto de dados iris e utilizar o autotype.

In [6]:
dados2 = Queryverse.load("iris.csv") |> DataFrame
dados2 = coerce(dados2, autotype(dados2))
schema(dados2)

┌─────────────┬───────────────────────────┬───────────────┐
│ _.names     │ _.types                   │ _.scitypes    │
├─────────────┼───────────────────────────┼───────────────┤
│ sepallength │ Float64                   │ Continuous    │
│ sepalwidth  │ Float64                   │ Continuous    │
│ petallength │ Float64                   │ Continuous    │
│ petalwidth  │ Float64                   │ Continuous    │
│ variety     │ CategoricalString{UInt32} │ Multiclass{3} │
└─────────────┴───────────────────────────┴───────────────┘
_.nrows = 150


Vemos que os dados já foram para o tipo científico adequado, ou seja, a função __autotype__ pode ser utilizada para uma preparação inicial e ,caso necessário, fazemos uso da função __coerce__ nos parâmetros que desejamos modificar.

Agora que já temos o noso conjunto de dados preparado, podemos separar em dados de entrada e dados de saída, para isso fazemos uso da função __unpack__. Essa função vai separar os dados em diferentes conjuntos, no caso abaixo, o primeiro conjunto vai ser a coluna de variety e o segundo vai ser todo o conjunto de dados menos a coluna variety.

In [7]:
y, x = unpack(dados, ==(:variety), colname -> true);

In [8]:
y[1:6]

6-element CategoricalArray{String,1,UInt32}:
 "Setosa"
 "Setosa"
 "Setosa"
 "Setosa"
 "Setosa"
 "Setosa"

In [9]:
first(x,6)

,sepallength,sepalwidth,petallength,petalwidth
,Float64,Float64,Float64,Float64
1,5.1,3.5,1.4,0.2
2,4.9,3.0,1.4,0.2
3,4.7,3.2,1.3,0.2
4,4.6,3.1,1.5,0.2
5,5.0,3.6,1.4,0.2
6,5.4,3.9,1.7,0.4


Suponhamos agora que desejamos dividir em dados de treino e teste, podemos fazer o uso da função __partition__ do pacote __MLJ__ para isso. Basicamente ela vai dividir um vetor em conjuntos, então podemos usar os índices do nosso vetor de dado como entrada.

In [10]:
treino, teste = partition(eachindex(y), 0.7, shuffle=true)

([37, 86, 58, 128, 108, 14, 46, 127, 132, 105  …  115, 65, 101, 4, 57, 34, 20, 1, 140, 50], [66, 113, 18, 19, 96, 23, 135, 29, 71, 15  …  16, 81, 73, 97, 9, 2, 124, 42, 94, 68])

com isso criamos dois vetores com índices dos dados que serão usados para treino e para teste, assim nosso dados de treino são dados por

In [11]:
y[treino], x[treino,:];

e os nosso dadso de teste são dados por

In [12]:
y[teste], x[teste,:];

É possível ainda fazer uma descrição de um conjunto de dados através do comando __describe__

In [13]:
describe(dados)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,sepallength,5.84333,4.3,5.8,7.9,,,Float64
2,sepalwidth,3.05733,2.0,3.0,4.4,,,Float64
3,petallength,3.758,1.0,4.35,6.9,,,Float64
4,petalwidth,1.19933,0.1,1.3,2.5,,,Float64
5,variety,,Setosa,,Virginica,3,,CategoricalString{UInt32}


Mais detalhes sobre dados pode ser visto na [documentação de exemplos](https://alan-turing-institute.github.io/MLJTutorials) do pacote __MLJ__, essa documentação deve ser usada também para entender outros aspectos de uso do pacote.